# SQL Functions, based `OllamaFunctions`

In [1]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [2]:
llama3 = OllamaFunctions(model="llama3:8b", temperature=0)
llama3_inst = OllamaFunctions(model="llama3:instruct", temperature=0)
llama3_inst_q8 = OllamaFunctions(model="llama3:8b-instruct-q8_0", temperature=0)
llama3_inst_fp16 = OllamaFunctions(model="llama3:8b-instruct-fp16", temperature=0)

In [14]:
llama3 = llama3.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

NotImplementedError: 

*The update is available on github, but has not yet been released.. waiting for